In [3]:
!pip install pandas
!pip install pyspark
!pip install pyarrow


[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [4]:
from pyspark.sql import SparkSession
 
# Building the SparkSession and name 
# it :'pandas to spark'
spark = SparkSession.builder.appName(
  "pandas to spark").getOrCreate()

spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

25/01/29 21:36:08 WARN Utils: Your hostname, Maazs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.2.24 instead (on interface en0)
25/01/29 21:36:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/29 21:36:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
import pandas as pd

data = [[1, 10, 5, 10, 10], [2, 20, 20, 20, 20], [3, 10, 30, 20, 20], [4, 10, 40, 40, 40]]
insurance = pd.DataFrame(data, columns=['pid', 'tiv_2015', 'tiv_2016', 'lat', 'lon']).astype({'pid':'Int64', 'tiv_2015':'Float64', 'tiv_2016':'Float64', 'lat':'Float64', 'lon':'Float64'})

In [6]:
insurance = spark.createDataFrame(insurance)
insurance.show()

+---+--------+--------+----+----+
|pid|tiv_2015|tiv_2016| lat| lon|
+---+--------+--------+----+----+
|  1|    10.0|     5.0|10.0|10.0|
|  2|    20.0|    20.0|20.0|20.0|
|  3|    10.0|    30.0|20.0|20.0|
|  4|    10.0|    40.0|40.0|40.0|
+---+--------+--------+----+----+



In [21]:
from pyspark.sql.functions import count, col, sum

investment = insurance \
            .groupby('tiv_2015') \
            .agg(count('pid').alias('count')) \
            .where('count > 1')

location = insurance \
            .groupby(['lat', 'lon']) \
            .agg(count('pid').alias('count')) \
            .where('count == 1')

report = insurance \
         .where(col('tiv_2015').isin(investment.toPandas()['tiv_2015'].to_list()) \
                & col('lat').isin(location.toPandas()['lat'].to_list()) \
                & col('lon').isin(location.toPandas()['lon'].to_list())   ) \
         .agg(sum('tiv_2016').alias('tiv_2016'))

report.show()

+--------+
|tiv_2016|
+--------+
|    45.0|
+--------+

